# B-state energy levels 
Calculating the energy levels and eigenstates of the B-state Hamiltonian

Import the relevant packages and Hamiltonians

In [1]:
import numpy as np
import pickle
import sys
import sympy

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state, find_state_idx_from_state, find_closest_state

with open("B_hamiltonians_symbolic.py",'rb') as f:
    hamiltonians = pickle.load(f)

def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
Jmin = 1
Jmax = 4 # max J value in Hamiltonian
#Jmax = 6
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation


Omegas = [-1, 1]
# QN = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2, Omega)
#       for J  in ni_range(Jmin, Jmax+1)
#       for Omega in Omegas
#       for mJ in ni_range(-J,J+1)
#       for m1 in ni_range(-I_Tl,I_Tl+1)
#       for m2 in ni_range(-I_F,I_F+1)
#      ]

QN = []
for Omega in Omegas:
    for J in np.arange(Jmin, Jmax+1):
        for m1 in np.arange(-I_Tl,I_Tl+1):
            for m2 in np.arange(-I_F,I_F+1):
                mJ = 0-m1-m2

                if np.abs(mJ) <= J:
                    QN.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2, Omega))


EOFError: Ran out of input

In [ ]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c1'),
    sympy.symbols('c_Tl_p'),
    *sympy.symbols('gL gS')
]

lambdified_hamiltonians = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians.items()
}

Insert the numerical constants:

In [ ]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c1 = -7.83e6
c_Tl_p = 11.17e6
gL = 1
gS = 2

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c1,
        c_Tl_p,
        gL, gS
    )
    for H_name, H_fn in lambdified_hamiltonians.items()
}



Check that Hamiltonian is hermitian and diagonalize the Hamiltonian

In [2]:
Hff = H["Hrot"]+H["H_mhf_Tl"]+H["H_mhf_F"]+H["Hc1"]+H["H_LD"]+H["H_c1_p"]
print(np.allclose(Hff, Hff.conj().T))
D, V = np.linalg.eigh(Hff)

NameError: name 'H' is not defined

In [31]:
states = matrix_to_states(V,QN)

In [32]:
for state in states[:8]:
    print("")
    state.remove_small_components().transform_to_coupled().print_state(tol=0.01)


-0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7071+0.0000j x |J = 1, F1 = 1/2, F = 0, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

-0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
-0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = -1, P = None>
+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0142+0.0000j x |J = 1, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>
+0.0126+0.0000j x |J = 2, F1 = 3/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1, P = None>

+0.7069+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, 

In [33]:
D

array([-1.44946348e+09, -1.43344748e+09, -8.85785272e+08, -8.69797942e+08,
        1.26311663e+10,  1.26488710e+10,  1.29479952e+10,  1.29659700e+10,
        3.96187852e+10,  3.96326908e+10,  3.98480859e+10,  3.98622043e+10,
        4.02995954e+10,  4.03138940e+10,  4.08814646e+10,  4.08955035e+10,
        7.98667511e+10,  7.98718302e+10,  8.00470530e+10,  8.00523005e+10,
        8.04098559e+10,  8.04439801e+10,  8.07542992e+10,  8.07882157e+10,
        1.33876561e+11,  1.33935533e+11,  1.34119936e+11,  1.34178729e+11,
        1.36504535e+11,  1.36530679e+11,  1.36689512e+11,  1.36715632e+11])

In [34]:
np.diff(D)

array([1.60160000e+07, 5.47662204e+08, 1.59873304e+07, 1.35009642e+10,
       1.77047268e+07, 2.99124178e+08, 1.79748440e+07, 2.66528152e+10,
       1.39055564e+07, 2.15395145e+08, 1.41183892e+07, 4.37391096e+08,
       1.42986036e+07, 5.67570529e+08, 1.40389616e+07, 3.89712476e+10,
       5.07910788e+06, 1.75222747e+08, 5.24749156e+06, 3.57555410e+08,
       3.41242492e+07, 3.10319041e+08, 3.39165140e+07, 5.30883451e+10,
       5.89717953e+07, 1.84403537e+08, 5.87924369e+07, 2.32580661e+09,
       2.61440715e+07, 1.58832527e+08, 2.61200000e+07])

## $\Omega$-splittings
Calculating energy differences between states of opposite parity

state = (J,F1,F)

state1 = (1,1/2,0)

state2 = (1,1/2,1)

state3 = (1,3/2,1)

state4 = (1,3/2,2)

state5 = (2,5/2,2)

state6 = (2,5/2,3)

state7 = (2,3/2,1)

state8 = (2,3/2,2)

In [35]:
#Define the states of interest (approximately)
state1e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state1f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=0, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state2e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state2f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 1/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state3e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state3f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state4e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state4f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 1, I1 = 1/2, I2 = 1/2, Omega = -1))

state5e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state5f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state6e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state6f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=3, mF = 0, F1 = 5/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state7e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state7f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=1, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state8e_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) +
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

state8f_appr = 1/np.sqrt(2)* ( 1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = +1) -
                                    1*CoupledBasisState(F=2, mF = 0, F1 = 3/2, J = 2, I1 = 1/2, I2 = 1/2, Omega = -1))

E1 = D[find_state_idx_from_state(Hff,state1e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state1f_appr,QN)]/1e6
print(E1)

E2 = D[find_state_idx_from_state(Hff,state2e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state2f_appr,QN)]/1e6
print(E2)

E3 = D[find_state_idx_from_state(Hff,state3e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state3f_appr,QN)]/1e6
print(E3)

E4 = D[find_state_idx_from_state(Hff,state4e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state4f_appr,QN)]/1e6
print(E4)

E5 = D[find_state_idx_from_state(Hff,state5e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state5f_appr,QN)]/1e6
print(E5)

E6 = D[find_state_idx_from_state(Hff,state6e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state6f_appr,QN)]/1e6
print(E6)

E7 = D[find_state_idx_from_state(Hff,state7e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state7f_appr,QN)]/1e6
print(E7)

E8 = D[find_state_idx_from_state(Hff,state8e_appr,QN)]/1e6 - D[find_state_idx_from_state(Hff,state8f_appr,QN)]/1e6
print(E8)

16.015999999999394
15.98733042714207
-17.704726798659976
-17.9748439801333
-13.905556383346266
-14.118389242095873
14.29860362848558
14.038961602200288


In [36]:
def calculate_splitting(state1,state2):
    return D[find_state_idx_from_state(Hff,state2,QN)]/1e6 - D[find_state_idx_from_state(Hff,state1,QN)]/1e6

## Splittings


In [37]:
ap = calculate_splitting(state1f_appr,state2f_appr)
print("a_+ = {:.2f} MHz".format(ap))

am = calculate_splitting(state1e_appr,state2e_appr)
print("a_- = {:.2f} MHz".format(am))

bp = calculate_splitting(state2f_appr,state3f_appr)
print("b_+ = {:.2f} MHz".format(bp))

bm = calculate_splitting(state2e_appr,state3e_appr)
print("b_- = {:.2f} MHz".format(bm))

cp = calculate_splitting(state3f_appr,state4f_appr)
print("c_+ = {:.2f} MHz".format(cp))

cm = calculate_splitting(state3e_appr,state4e_appr)
print("c_- = {:.2f} MHz".format(cm))

omega1 = calculate_splitting(state2f_appr,state2e_appr)
print("omega1 = {:.2f} MHz".format(omega1))

omega2 = calculate_splitting(state3f_appr,state3e_appr)
print("omega2 = {:.2f} MHz".format(omega2))

a_+ = 563.68 MHz
a_- = 563.65 MHz
b_+ = 13534.66 MHz
b_- = 13500.96 MHz
c_+ = 317.10 MHz
c_- = 316.83 MHz
omega1 = 15.99 MHz
omega2 = -17.70 MHz
